In [44]:
from pymatgen.core import Lattice, Structure 
import numpy as np 
from pymatgen.core.composition import Element, Composition
from pymatgen.core.periodic_table import Specie
import math
import random
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Potcar,Incar
from pymatgen.io.vasp.outputs import Outcar 
from pymatgen.entries.computed_entries import ComputedStructureEntry
import os
import clease
from clease.settings import Concentration
import numpy as np
from clease.settings import CEBulk
from clease.structgen import NewStructures
from ase.db import connect
from ase.io import write
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter
import json
# now lets make the computed entries from chgnet potential 
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer

# Functions 

In [45]:
def create_cca_primitive(atom_dict, a, prim = True):
    """
    Create a CCA (Complex Concentrated Alloy) primitive structure using the given atom dictionary and lattice constant.

    Parameters:
        atom_dict (dict): A dictionary containing the count of each atom in the supercell you eventually want.
        a (float): The lattice constant of the cubic cell.

    Returns:
        pymatgen.core.structure.Structure: The CCA supercell structure.

    """
    comp_list = list(atom_dict.keys())

    elements = [{element: count/sum(atom_dict.values())} for element, count in atom_dict.items()]
    c_v = 1 - sum(elements[0].values())
    elements[0][comp_list[0]] = c_v

    merged_elements = {k:v for element in elements for k,v in element.items()}
    elements = [merged_elements]
    #print(elements)
    direct_coords = [[0,0,0]]
    if prim:
        test_bcc = Structure(Lattice.cubic(a), comp_list, direct_coords)
    else:
        test_bcc = Structure.from_spacegroup("Im-3m",Lattice.cubic(a),elements,direct_coords)
    return test_bcc

def closest_composition(comp, num_atoms):
    """
    Calculates the closest composition of elements to a given number of atoms.

    Parameters:
    comp (dict): A dictionary representing the composition of elements, where the keys are the element symbols and the values are the fractions.
    num_atoms (int): The total number of atoms.

    Returns:
    tuple: A tuple containing two dictionaries. The first dictionary represents the number of atoms for each element, and the second dictionary represents the actual fractions of each element.

    Example:
    comp = {'H': 0.5, 'O': 0.5}
    num_atoms = 10
    atoms, actual_fractions = closest_composition(comp, num_atoms)
    print(atoms)  # Output: {'H': 5, 'O': 5}
    print(actual_fractions)  # Output: {'H': 0.5, 'O': 0.5}
    """
    # Normalize the composition fractions
    total_fraction = sum(comp.values())
    normalized_comp = {element: fraction / total_fraction for element, fraction in comp.items()}
    
    # Initial calculation of atoms, rounding down to avoid exceeding num_atoms
    atoms = {element: int(normalized_comp[element] * num_atoms) for element in normalized_comp}
    
    # Calculate shortfall from num_atoms after initial rounding
    shortfall = num_atoms - sum(atoms.values())
    
    # Distribute shortfall by rounding up closest elements that were previously rounded down
    fractions_sorted_by_residual = sorted(normalized_comp, key=lambda x: normalized_comp[x] * num_atoms - atoms[x], reverse=True)
    for i in range(shortfall):
        atoms[fractions_sorted_by_residual[i]] += 1
    
    # Ensure no adjustments result in negative values
    # This step may not be strictly necessary with the above logic, but is a good safeguard
    for element in atoms:
        atoms[element] = max(0, atoms[element])
    
    # Recalculate the actual fractions to ensure they sum to 1
    actual_fractions = {element: atoms[element] / num_atoms for element in atoms}
    
    return atoms, actual_fractions

def create_random_supercells(composition,alat,supercell_size,db_name,num_structures):
    # A_eq is the identity matrix with a size equal to the number of elements in the composition, i.e len(composition.keys())
    A_eq = np.eye(len(composition.keys()))
    b_eq = list(composition.values())
    conc = Concentration(basis_elements=[list(composition.keys())],
                         A_eq = A_eq,
                         b_eq = b_eq)
    
    settings = CEBulk(crystalstructure='bcc',
                      a=alat,
                      size=[supercell_size,supercell_size,supercell_size],
                      concentration=conc,
                      db_name=db_name,
                      max_cluster_dia=[6.0, 4.5, 4.5])
    


    ns = NewStructures(settings, generation_number=1,
                    struct_per_gen=num_structures)
    ns.generate_random_structures()

def connect_ase_db(db_name):
    from ase.db import connect
    atoms = connect(db_name)
    print(len(atoms))
    for idx in range(len(atoms)):
        curr_struct = atoms.get(id=idx+1).toatoms()
        print(curr_struct)
    return atoms

        
def ase_db_to_pymatgen(db_name,output_file=None):

    data = {}
    Adaptor = AseAtomsAdaptor()
    atoms = connect(db_name) 

    for idx in range(2,len(atoms)+1):
        
        curr_struct = atoms.get(id=idx).toatoms()
        
        curr_pymatgen = Adaptor.get_structure(curr_struct)
        
        
        trans = PerturbStructureTransformation(distance=0.1, min_distance=0.01)
        distorted_struct = trans.apply_transformation(curr_pymatgen)
        
        data.update({f'{idx}':distorted_struct.as_dict()})
        
    if output_file is not None:
        with open(output_file, 'w') as f:
            json.dump(data, f,)
    else:
        return data
        
def generate_random_supercells(composition, num_structures, lattice_parameter=3.01, supercell_size=4,supercell_type='cubic', seed=42):
    random.seed(seed) 
    supercells = []

    comp_list = [key for key in composition]
    for _ in range(num_structures):
        # Create a bulk V crystal with the specified lattice parameter
        #prim_cell = bulk(comp_list[0], cubic=True, a=lattice_parameter)
        if supercell_type == 'cubic':
            prim_cell = Structure(Lattice.cubic(lattice_parameter), [comp_list[0],comp_list[0]], [[0, 0, 0],[0.5, 0.5, 0.5]])
        elif supercell_type == 'prim':
            prim_cell = Structure(Lattice.cubic(lattice_parameter), [comp_list[0]], [[0, 0, 0]])
        
        # make the supercell
        supercell = prim_cell * (supercell_size,supercell_size,supercell_size)

        # Create a list of all possible indices for the current element
        all_indices = list(range(len(supercell.sites)))

        for element, count in composition.items():
            # Randomly select 'count' indices for the current element
            selected_indices = random.sample(all_indices, count)
            
            # Change the selected indices to the current element
            for index in selected_indices:
                supercell.replace(index,element)
            
            # remove the selected_indices from the all_indices list
            all_indices = [index for index in all_indices if index not in selected_indices]
        
        supercell = supercell.get_sorted_structure()

        supercells.append(supercell)

    return supercells

def make_vasp_job(supercell, job_path, kpoints_params, incar_params=None):
    """
    Create a VASP job for the given supercell.

    Parameters:
        supercell (pymatgen.core.structure.Structure): The supercell for which to create the VASP job.
        job_path (str): The path to the directory where the VASP job should be created.
        incar_params (dict): A dictionary containing the INCAR parameters.
        kpoints_params (list): A tuple containing the kpoints parameters i.e (3,3,3)
        potcar_path (str): The path to the directory containing the POTCAR files.

    Returns:
        None

    """
    # make the POSCAR 
    poscar = Poscar(structure = supercell, comment = str(supercell.composition))
    
    # make the KPOINTS
    kpoints = Kpoints.gamma_automatic(kpts = kpoints_params)
    
    # make the POTCAR
    #potcar = Potcar(symbols=[str(e.symbol)+"_pv" for e in supercell.composition.elements], functional="PBE_64")

    symbols = []
    for e in supercell.composition.elements:
        try:
            symbols.append(str(e.symbol) + "_pv")
            potcar = Potcar(symbols=symbols, functional="PBE_64")
        except:
            symbols[-1] = str(e.symbol) + "_sv"
            potcar = Potcar(symbols=symbols, functional="PBE_64")
    
    potcar = Potcar(symbols=symbols, functional="PBE_64")
    
    # make the INCAR
    if incar_params is None:
        incar_params = {'ALGO':'Fast',
              'ENCUT': 360,
              'EDIFF': 1E-6,
              'EDIFFG': -0.01,
              'ISMEAR': 1,
              'SIGMA': 0.2,
              'LWAVE': '.FALSE.',
              'LCHARG': '.FALSE.',
              'LORBIT': 11,
              'LREAL': '.FALSE.',
              'NELM' : 100,
              'NELMIN' : 5,
              'NSW' : 200,
              'PREC' : 'Accurate',
              'IBRION' : 3,
              'SMASS' : 1.85,
              'POTIM' : 0.25,
              'ISIF' : 3,
              #'NBANDS' : supercell.num_sites*2*5, # assuming 5 electrons per atom
              }
    else:
        incar_params = incar_params
    incar = Incar(incar_params)

    # make slurm file 
    
    # write the files to the job_path
    poscar.write_file(filename=os.path.join(job_path, "POSCAR"))
    kpoints.write_file(filename=os.path.join(job_path, "KPOINTS"))
    potcar.write_file(filename=os.path.join(job_path, "POTCAR"))
    incar.write_file(filename=os.path.join(job_path, "INCAR"))

def make_slurm_file(job_path, i, time="24:00:00", nodes=1, num_gpus=1,omp_threads=1):
    """
    Create a SLURM job for the given VASP job.

    Parameters:
        job_path (str): The path to the directory containing the VASP job.
        job_name (str): The name of the job.
        time (str): The time the job should run for.
        nodes (int): The number of nodes the job should run on.
        ppn (int): The number of processors per node.
        queue (str): The queue to which the job should be submitted.

    Returns:
        None

    """
    slurm_file = open(os.path.join(job_path, "job.slurm"), "w")
    slurm_file.write("#!/bin/bash\n")
    slurm_file.write(f"#SBATCH --job-name=job_{i}\n")
    slurm_file.write("#SBATCH --output=std-out\n")
    slurm_file.write("#SBATCH --time=" + time + "\n")
    slurm_file.write("#SBATCH --nodes=" + str(nodes) + "\n")
    slurm_file.write(f"#SBATCH --ntasks-per-node={num_gpus}\n")
    slurm_file.write("#SBATCH --constraint=v100s\n")
    slurm_file.write(f"#SBATCH --gres=gpu:{num_gpus}\n")
    #slurm_file.write("#SBATCH --partition=" + queue + "\n")
    #slurm_file.write("module load vasp/5.4.4\n")
    if omp_threads > 1:
        slurm_file.write(f"export OMP_NUM_THREADS={omp_threads}\n")
    #slurm_file.write("export ")
    slurm_file.write("cd $SLURM_SUBMIT_DIR\n")
    slurm_file.write("/opt/nvidia/hpc_sdk/Linux_x86_64/22.5/comm_libs/mpi/bin/mpirun /home/myless/VASP/vasp.6.3.2/bin/vasp_std\n")
    slurm_file.write("exit\n")
    #slurm_file.write("mpirun vasp_std\n")
    slurm_file.close()
    
    #!/bin/bash # #SBATCH --job-name=neb_Cr9Ti10V44_Structure_Num_0_Vac_site_num_0_110 #SBATCH --output=std-out #SBATCH --ntasks-per-node=3 #SBATCH --nodes=1 #SBATCH --gres=gpu:3 #SBATCH --constraint=v100s #SBATCH --time=24:00:00 #SBATCH -p regular

def create_computed_entry_from_outcar(outcar,poscar=None):
    """
    Create a ComputedEntry from the given OUTCAR file."""
    # need to test this with an outcar 
    outcar = Outcar(outcar)
    energy = outcar.final_energy
    composition = outcar.structure.composition
    if Poscar is not None:
        poscar = Poscar.from_file(poscar)
        structure = poscar.structure
    else:
        structure = outcar.structure

    return ComputedStructureEntry(composition, energy, structure = structure)

def create_computed_entry_from_chgnet(structure, energy):
    composition = structure.composition
    return ComputedStructureEntry(composition=composition, energy = energy, structure=structure)
    
def relax_structures(supercells, potential_path,device='cpu',verbose=False):
    vac_pot = CHGNet.from_file(potential_path, use_device=device)
    entries = []
    relaxer = StructOptimizer(vac_pot, use_device=device)

    # If vcrti_generated_supercells is a dictionary, convert it to a list
    if isinstance(supercells, dict):
        supercells = list(supercells.values())
    print(len(supercells))
    for i, supercell in enumerate(supercells):
        if isinstance(supercell, dict):
            supercell = Structure.from_dict(supercell)
        relaxed_supercell = relaxer.relax(atoms=supercell, fmax=0.05, relax_cell=True, verbose=verbose)
        print("Finished Volumetric Relaxing Structure: ", i)
        final_result = relaxer.relax(atoms=relaxed_supercell['final_structure'], fmax=0.05, relax_cell=False, verbose=verbose)
        print("Final Energy: ", final_result['trajectory'].energies[-1])

        # make the computed entry
        entry = create_computed_entry_from_chgnet(supercell, final_result['trajectory'].energies[-1])
        entries.append(entry.as_dict())

    return entries

def generate_compositions(atom_dict, n, balance_element, seed=42):
    """
    Generate random compositions based on given atom_dict and balance_element.

    Parameters:
    - atom_dict (dict): A dictionary containing the elements as keys and their corresponding minimum and maximum values as values.
    - n (int): The number of compositions to generate.
    - balance_element (str): The element that needs to be balanced in the compositions.
    - seed (int): The seed value for the random number generator. Default is 42.

    Returns:
    - samples (list): A list of dictionaries representing the generated compositions.
    """

    random.seed(seed)
    samples = []

    for _ in range(n):
        sample = {}
        total_percentage = 0.0

        # Randomly sample for each element except the balance element
        for element, (min_val, max_val) in atom_dict.items():
            if element != balance_element:
                sample[element] = round(random.uniform(min_val, max_val), 4)
                total_percentage += sample[element]

        # Adjust if sum of other elements exceeds 1
        if total_percentage >= 1.0:
            # Scale down the sampled values proportionally if the total exceeds 100%
            scale_factor = (1.0 - atom_dict[balance_element][0]) / total_percentage
            for element in sample:
                sample[element] = round(sample[element] * scale_factor, 4)
            total_percentage = sum(sample.values())

        # Ensure the balance element's value makes the total sum 1
        sample[balance_element] = round(1.0 - total_percentage, 4)

        # Check if balance is less than its min value, adjust if necessary
        if sample[balance_element] < atom_dict[balance_element][0]:
            # This scenario should be rare with proper input ranges
            # An adjustment strategy might be needed depending on specific requirements
            raise ValueError(f"Cannot meet balance element '{balance_element}' minimum requirement without exceeding total of 1.0")

        samples.append(sample)

    return samples


# Run the Code

In [3]:
x_cr = 0.02
x_ti  = 0.02
x_w = 0.04
x_zr  = 0.01
x_ta = 0.005
x_v = 1 - x_cr - x_ti - x_w - x_zr - x_ta
supercell_size = 5
num_atoms = supercell_size**3 
num_cr  = math.ceil(num_atoms * x_cr)
num_ti = math.ceil(num_atoms * x_ti)
num_w  = math.ceil(num_atoms * x_w)
num_zr = math.ceil(num_atoms * x_zr)
num_ta = math.ceil(num_atoms * x_ta)
#num_v = 2*supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
num_v = supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti , 'Zr' : num_zr, 'W' : num_w, 'Ta' : num_ta}
#atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti}
num_structures = 30 
print(atom_dict)
db_name = '_'.join([f'{key}{value}' for key, value in atom_dict.items()]) + '.db'
print(db_name)


{'V': 111, 'Cr': 3, 'Ti': 3, 'Zr': 2, 'W': 5, 'Ta': 1}
V111_Cr3_Ti3_Zr2_W5_Ta1.db


In [46]:
# Example usage:
element_ranges = {'V': (0.1, 0.9), 'Cr': (0.01, 0.1), 'Ti': (0.01, 0.1), 'Zr': (0.01, 0.05), 'W': (0.01, 0.1)}
num_compositions = 20
compositions = generate_compositions(element_ranges, n = num_compositions,balance_element='V',seed=42)
for i, composition in enumerate(compositions, 1):
    print(f'Composition {i}: {composition}')

Composition 1: {'Cr': 0.0675, 'Ti': 0.0123, 'Zr': 0.021, 'W': 0.0301, 'V': 0.8691}
Composition 2: {'Cr': 0.0763, 'Ti': 0.0709, 'Zr': 0.0457, 'W': 0.0178, 'V': 0.7893}
Composition 3: {'Cr': 0.048, 'Ti': 0.0127, 'Zr': 0.0187, 'W': 0.0555, 'V': 0.8651}
Composition 4: {'Cr': 0.0124, 'Ti': 0.0279, 'Zr': 0.036, 'W': 0.059, 'V': 0.8647}
Composition 5: {'Cr': 0.0298, 'Ti': 0.063, 'Zr': 0.0424, 'W': 0.0106, 'V': 0.8542}
Composition 6: {'Cr': 0.0825, 'Ti': 0.0728, 'Zr': 0.0236, 'W': 0.024, 'V': 0.7971}
Composition 7: {'Cr': 0.0961, 'Ti': 0.0403, 'Zr': 0.0137, 'W': 0.0187, 'V': 0.8312}
Composition 8: {'Cr': 0.0863, 'Ti': 0.0643, 'Zr': 0.0423, 'W': 0.0757, 'V': 0.7314}
Composition 9: {'Cr': 0.0583, 'Ti': 0.0976, 'Zr': 0.0251, 'W': 0.0597, 'V': 0.7593}
Composition 10: {'Cr': 0.0846, 'Ti': 0.0657, 'Zr': 0.0445, 'W': 0.062, 'V': 0.7432}
Composition 11: {'Cr': 0.0734, 'Ti': 0.0141, 'Zr': 0.0191, 'W': 0.036, 'V': 0.8574}
Composition 12: {'Cr': 0.0172, 'Ti': 0.031, 'Zr': 0.014, 'W': 0.035, 'V': 0.9028}


In [4]:
new_atoms, new_atom_dict = closest_composition(atom_dict, num_atoms)



In [47]:
db_directory = '/Users/myless/Packages/structure_maker/VCrTiWZr_DBs'
vasp_job_path = '/Users/myless/Packages/structure_maker/CE_Vasp_Jobs_T3'
num_structures = 10
supercell_size = 4
num_atoms = supercell_size**3
structures = []
k = 0 
if not os.path.exists(vasp_job_path):
    os.makedirs(vasp_job_path)
for i, composition in enumerate(compositions):
    new_atoms, new_atom_dict = closest_composition(composition, num_atoms)
    print(new_atom_dict)
    db_name = '_'.join([f'{key}{value}' for key, value in new_atom_dict.items()]) + '.db'
    generate_supercells = create_random_supercells(new_atom_dict, 3.01, supercell_size, os.path.join(db_directory,db_name), num_structures)
    generated_structures = ase_db_to_pymatgen(os.path.join(db_directory,db_name))
    for j, supercell in enumerate(generated_structures.values()):
        if isinstance(supercell, dict):
            structures.append(supercell)
        else:
            structures.append(supercell.as_dict())
            
        job_path = os.path.join(vasp_job_path, f'job_{k}')
        os.makedirs(job_path, exist_ok=True)
        if isinstance(supercell, dict):
            supercell = Structure.from_dict(supercell)
            supercell = supercell.get_sorted_structure()
        make_vasp_job(supercell.get_sorted_structure(), job_path, kpoints_params=(3,3,3), incar_params=None)
        make_slurm_file(job_path, k, num_gpus=1,omp_threads=10)
        print(f'Job {k} made')
        k += 1 
    
# write to the db_directory as a json 
with open(os.path.join(db_directory, 'vcrtiwzr_t3_structures.json'), 'w') as f:
    json.dump(structures, f)

{'Cr': 0.0625, 'Ti': 0.015625, 'Zr': 0.015625, 'W': 0.03125, 'V': 0.875}
Job 0 made
Job 1 made
Job 2 made
Job 3 made
Job 4 made
Job 5 made
Job 6 made
Job 7 made
Job 8 made
Job 9 made
{'Cr': 0.078125, 'Ti': 0.078125, 'Zr': 0.046875, 'W': 0.015625, 'V': 0.78125}
Job 10 made
Job 11 made
Job 12 made
Job 13 made
Job 14 made
Job 15 made
Job 16 made
Job 17 made
Job 18 made
Job 19 made
{'Cr': 0.046875, 'Ti': 0.015625, 'Zr': 0.015625, 'W': 0.0625, 'V': 0.859375}
Job 20 made
Job 21 made
Job 22 made
Job 23 made
Job 24 made
Job 25 made
Job 26 made
Job 27 made
Job 28 made
Job 29 made
{'Cr': 0.015625, 'Ti': 0.03125, 'Zr': 0.03125, 'W': 0.0625, 'V': 0.859375}
Job 30 made
Job 31 made
Job 32 made
Job 33 made
Job 34 made
Job 35 made
Job 36 made
Job 37 made
Job 38 made
Job 39 made
{'Cr': 0.03125, 'Ti': 0.0625, 'Zr': 0.046875, 'W': 0.015625, 'V': 0.84375}
Job 40 made
Job 41 made
Job 42 made
Job 43 made
Job 44 made
Job 45 made
Job 46 made
Job 47 made
Job 48 made
Job 49 made
{'Cr': 0.078125, 'Ti': 0.078125,

In [5]:
generated_supercells = create_random_supercells(new_atom_dict, 3.01, supercell_size, db_name, num_structures)
# I want the db name to be the composition of the supercell coming from atom dict, i.e key0value0_key1value1_key2value2_keynvaluen.db
generated_structures = ase_db_to_pymatgen(db_name=db_name)

In [16]:
import json 
# write all the structures to a json 
structures = []
structures_name = 'structures_' + db_name.split('.')[0] + '.json'
with open(structures_name, 'w') as f:
    for i, supercell in enumerate(generated_structures.values()):
        if isinstance(supercell, dict):
            structures.append(supercell)
        else:
            structures.append(supercell.as_dict())
    json.dump(structures, f)
        #f.write(supercell.to(fmt='poscar'))
        #f.write('\n')

In [17]:
pot_path = '/home/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'
entries = relax_structures(generated_structures, pot_path, device='cuda', verbose=True)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda:0
30
      Step     Time          Energy          fmax
FIRE:    0 20:47:11    -1134.681821        2.350445
FIRE:    1 20:47:11    -1135.030508        1.893607
FIRE:    2 20:47:11    -1135.588527        1.235785
FIRE:    3 20:47:11    -1136.181593        0.824988
FIRE:    4 20:47:11    -1136.692882        0.648406
FIRE:    5 20:47:11    -1137.030125        0.424253
FIRE:    6 20:47:11    -1137.205720        0.719424
FIRE:    7 20:47:11    -1137.267351        0.822172
FIRE:    8 20:47:11    -1137.273550        0.806053
FIRE:    9 20:47:11    -1137.285590        0.772258
FIRE:   10 20:47:11    -1137.302637        0.717118
FIRE:   11 20:47:12    -1137.323380        0.628722
FIRE:   12 20:47:12    -1137.346506        0.536668
FIRE:   13 20:47:12    -1137.370706        0.443685
FIRE:   14 20:47:12    -1137.394428        0.332003
FIRE:   15 20:47:12    -1137.418270        0.206961
FIRE:   16 20:47:12    -1137.439966    

In [13]:
#main_vasp_job_path = '/Users/myless/Dropbox (MIT)/Research/2024/IAP_2024/Structure_Making/optimized_clease_vcrti_structures/'
main_vasp_job_path = './test_gpu_mem/'
# make the directory for the jobs if it exists already, remove
if os.path.exists(main_vasp_job_path):
    import shutil
    shutil.rmtree(main_vasp_job_path)
os.makedirs(main_vasp_job_path, exist_ok=True)
for i, supercell in enumerate(generated_structures.values()):
    job_path = os.path.join(main_vasp_job_path, f'job_{i}')
    os.makedirs(job_path, exist_ok=True)
    if isinstance(supercell, dict):
        supercell = Structure.from_dict(supercell)
    make_vasp_job(supercell, job_path, kpoints_params=(3,3,3), incar_params=None)
    make_slurm_file(job_path, i, num_gpus=1)
    print(f'Job {i} made')

Job 0 made
Job 1 made
Job 2 made
Job 3 made
Job 4 made
Job 5 made
Job 6 made
Job 7 made
Job 8 made
Job 9 made
Job 10 made
Job 11 made
Job 12 made
Job 13 made
Job 14 made
Job 15 made
Job 16 made
Job 17 made
Job 18 made
Job 19 made
Job 20 made
Job 21 made
Job 22 made
Job 23 made
Job 24 made
Job 25 made
Job 26 made
Job 27 made
Job 28 made
Job 29 made


In [35]:
print(vcrti_generated_supercells)

{'2': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], 'pbc': (True, True, True), 'a': 10.392304845413264, 'b': 10.392304845413264, 'c': 10.392304845413264, 'alpha': 109.47122063449069, 'beta': 109.47122063449069, 'gamma': 109.47122063449069, 'volume': 864.0}, 'properties': {}, 'sites': [{'species': [{'element': 'V', 'occu': 1}], 'abc': [0.7474926056280402, 0.002486764353095562, 0.0021711782510697697], 'xyz': [-4.457007978143249, 4.483062117156086, 4.486849150380396], 'properties': {'tags': 0}, 'label': 'V'}, {'species': [{'element': 'V', 'occu': 1}], 'abc': [0.5029891329716127, 0.009875914261848265, 0.00458912319922788], 'xyz': [-2.9311445730632197, 2.9862140514539544, 3.0496555442053985], 'properties': {'tags': 0}, 'label': 'V'}, {'species': [{'element': 'V', 'occu': 1}], 'abc': [0.7504074237844163, 0.25295414828943336, 0.0006565557715262581], 'xyz': [-2.9807803183407393, 2.9886589

In [36]:
potential_path = '/Users/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'

entries = relax_structures(generated_supercells,potential_path=potential_path,device='mps',verbose=True)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps
30
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 14:59:28     -569.518066*       0.6284
FIRE:    1 14:59:29     -569.565369*       0.5979
FIRE:    2 14:59:29     -569.652222*       0.5363
FIRE:    3 14:59:30     -569.764282*       0.4536
FIRE:    4 14:59:30     -569.885315*       0.3780
FIRE:    5 14:59:31     -569.999634*       0.2887
FIRE:    6 14:59:32     -570.092468*       0.2117
FIRE:    7 14:59:32     -570.152466*       0.1229
FIRE:    8 14:59:32     -570.177246*       0.1001
FIRE:    9 14:59:33     -570.162842*       0.1690
FIRE:   10 14:59:33     -570.164185*       0.1666
FIRE:   11 14:59:33     -570.166626*       0.1618
FIRE:   12 14:59:33     -570.170166*       0.1540
FIRE:   13 14:59:34     -570.174561*       0.1429
FIRE:   14 14:59:34     -570.179504*       0.1287
FIRE:   15 14:59:35     -570.184753*       0.1115
FIRE:   16 14:59:35     -5

In [20]:
# write the entries 
with open(db_name.split('.')[0] +'_entries.json', 'w') as f:
    json.dump(entries, f)

In [72]:
test_entry = create_computed_entry_from_chgnet(relaxed_supercell['final_structure'], relaxed_supercell['trajectory'].energies[-1])
print(test_entry.as_dict())

{'@module': 'pymatgen.entries.computed_entries', '@class': 'ComputedStructureEntry', 'energy': -564.9136962890625, 'composition': {'Ti': 3.0, 'V': 58.0, 'Cr': 3.0}, 'entry_id': None, 'correction': 0.0, 'energy_adjustments': [], 'parameters': {}, 'data': {}, 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[9.816883288100458, -0.045030822457039724, -0.2707692631636627], [-0.047158841067081375, 9.197235696296065, 0.05166922563444473], [-0.26635979526982545, 0.04595135277667574, 9.807948992369784]], 'pbc': (True, True, True), 'a': 9.820719997079205, 'b': 9.197501732448384, 'c': 9.811672767896699, 'alpha': 89.40193540084451, 'beta': 93.13672659512305, 'gamma': 90.56525454926621, 'volume': 884.8357032215285}, 'properties': {}, 'sites': [{'species': [{'element': 'Ti', 'occu': 1}], 'abc': [-0.010048356212177116, 0.024999448755645946, 0.16055000354673765], 'xyz': [-0.1425865512781129, 0.23775579807899946, 1.5786787336772603], 'prope

In [41]:
x_cr = 0.02
x_ti  = 0.03
x_w = 0.05
x_zr  = 0.02
x_ta = 0.01
x_v = 1 - x_cr - x_ti - x_w - x_zr - x_ta
supercell_size = 4
num_atoms = 2*supercell_size**3 
num_cr  = math.ceil(num_atoms * x_cr)
num_ti = math.ceil(num_atoms * x_ti)
num_w  = math.ceil(num_atoms * x_w)
num_zr = math.ceil(num_atoms * x_zr)
num_ta = math.ceil(num_atoms * x_ta)
num_v = 2*supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti , 'Zr' : num_zr, 'W' : num_w, 'Ta' : num_ta}
num_structures = 10 
print(atom_dict)
generated_supercells = generate_random_supercells(atom_dict, num_structures, supercell_size=supercell_size)


{'V': 109, 'Cr': 3, 'Ti': 4, 'Zr': 3, 'W': 7, 'Ta': 2}
